# Image similarity model


In [1]:
from tensorflow.keras.layers import Flatten, Dense, Input,concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import tensorflow as tf
import cv2
from scipy.spatial import distance
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
# TODO Import Tobis module get_test_dataframe

### Import VGG16 model from Tensorflow

In [2]:
vgg16 = tf.keras.applications.VGG16(weights='imagenet', include_top=True, pooling='max', input_shape=(224, 224, 3))

553467904/553467096 [==============================] - 21s 0us/step


In [3]:
basemodel = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)

In [4]:
#basemodel.summary()

### Vectorizer for images

In [4]:
def get_feature_vector(img):
 img1 = cv2.resize(img, (224, 224))
 feature_vector = basemodel.predict(img1.reshape(1, 224, 224, 3))
 return feature_vector

### Calculate similarity function

In [5]:
def calculate_similarity(vector1, vector2):
 return distance.cosine(vector1, vector2)

### Create vectors for all images in local folder

In [7]:
# directory = '/home/beres/code/tklein98/crate_scanner/notebooks/images/'
# vectors = []

# for filename in os.listdir(directory):
#     img = cv2.imread(f'images/{filename}')
#     img1 = get_feature_vector(img)
#     vectors.append([filename, img1.copy()])

### The vector file can be saved and retrieved as follows;

In [ ]:
# The vectors array has been saved to the notebooks folder in the project repo 
# array = np.array(vectors)
# np.save('array', array)

In [14]:
## Load saved file using the following code; x_loaded = np.load(path/'x.npy')
vectors = np.load('array.npy', allow_pickle = True)

vectors

array([['I Want to See the Bright Lights Tonight Richard & Linda Thompson.jpg',
        array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)],
       ['Shoot Out the Lights Richard & Linda Thompson.jpg',
        array([[0.        , 0.        , 0.        , ..., 0.        , 0.32527405,
        1.7335162 ]], dtype=float32)],
       ['Child Is Father to the Man Blood, Sweat & Tears.jpg',
        array([[1.1589096, 0.       , 0.       , ..., 0.       , 4.6766176,
        1.6251488]], dtype=float32)],
       ['Late Registration Kanye West.jpg',
        array([[1.4965571, 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)],
       ['Rocks Aerosmith.jpg',
        array([[0.        , 0.33209676, 0.        , ..., 0.        , 0.5972106 ,
        0.02758223]], dtype=float32)],
       ['Music of My Mind Stevie Wonder.jpg',
        array([[0.       , 0.       , 0.4285223, ..., 0.       , 0.       ,
        0.       ]], dtype=float32)],
       ['The Joshua Tree U2.j

### Import test image and find similar image

In [12]:
img1 = cv2.imread("test_images/Arcade Fire_Funeral_38.jpg")

f1 = get_feature_vector(img1)
comparison = []

for vector in vectors:
    for i in vector[1]:
        comparison.append(calculate_similarity(f1, i))
        

### Retrieve details for closest match

In [11]:
index = comparison.index(min(comparison))
album = vectors[index][0]
print(album)
album = cv2.imread(f'images/{album}')[:,:,::-1]
plt.imshow(album)

Funeral Arcade Fire.jpg


TypeError: 'NoneType' object is not subscriptable

### Loop over test images and find matches

In [17]:
# This filepath is local, Beres: 1st, Tobias: 2nd

# Beres
directory = '/home/beres/code/tklein98/crate_scanner/notebooks/test_images/subset/'

# Tobi
# directory = "/Users/Tobias/Desktop/project_weeks/Pictures/"

comparisons = []
import ipdb


for filename in os.listdir(directory):
    img = cv2.imread(f'test_images/subset/{filename}')
    img1 = get_feature_vector(img)
    comparison = {}
    counter = 0
    for vector in vectors:
        for j in vector[1]:
            #ipdb.set_trace()
            if len(comparison) == 0:
                #ipdb.set_trace()
                comparison[filename] = (counter, calculate_similarity(img1, j))
            elif comparison[filename][1] > calculate_similarity(img1, j):
                #ipdb.set_trace()
                comparison[filename] = (counter, calculate_similarity(img1, j))
            counter += 1
    comparisons.append(comparison)


error: OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-39p1qqfs/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [18]:
comparisons

[]

In [16]:
vectors[264][0]

'Blonde on Blonde Bob Dylan.jpg'

In [20]:
# Caution not the best way for cleaning!!!

whitelist = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
cleaned_string = ''.join(filter(whitelist.__contains__, vectors[264][0]))
cleaned_string = cleaned_string[:-3].lower()

cleaned_string

'blondeonblondebobdylan'

In [ ]:
#TODO: Retrieve the testing images with their labels with model from tobi called get_testing_dataframe, syntax artistalbum all lowercase no spaces

# TO retrieve the dataframe: simply do get_testing_dataframe(directory)

# df_all_spotify_images is the master table

In [ ]:
# Important: change os.listdir(directory) later to the dataframe of get_testing_dataframe

In [ ]:
# directory is the length of the testing images set


def accuracy_model(testing_dataframe):
# introduce score to calculate the accuracy 
    score = 0 

    for i in range(len(os.listdir(directory))):
        # look in comparisons, take the index of the image, get the key value through testing_dataframe['filename'][i] and get the first value of the tuple (index)
        index_matched_image = comparisons[i][testing_dataframe['filename'][i]][0]
        # Get the predicted album from all the vectors with the defined index, this assumes that the label is stored in the first place or 0
        predicted_album = df_all_spotify_images['col_artist_album'][index_matched_image]
        
        # TODO: Clean the album string as the output will be something like this: 'Blonde on Blonde Bob Dylan'
        predicted_album_cleaned = 
        # If the predicted label matches the label from the labeled Dataframe with the test images, increase score by one
        if testing_dataframe['label'][i] == predicted_album_cleaned:
            score += 1
            
    return score/len(os.listdir(directory))
        
        